In [1]:
# XGBoost on Otto dataset, Tune n_estimators
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier 



from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot

import os
import time
from datetime  import datetime

In [2]:
#strategyName='Short'
#pahtname=r'D:\DataSets\ML_LS\ML-Short_S50M15_07To2Q20-Train_Norm.csv'


pahtname=r'D:\DataSets\ML-Long_S50M15_07To1Q21.csv'

filepath= os.path.abspath(pahtname)

dataset =pd.read_csv(filepath,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
dataset.drop(columns=['open','high','low','close'],inplace=True)

dataset.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75006 entries, 2007-01-03 09:45:00 to 2021-03-31 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             75006 non-null  float64
 1   indy_ma-1100            75006 non-null  float64
 2   indy_hh-550             75006 non-null  float64
 3   indy_ll-550             75006 non-null  float64
 4   indy_mid-550            75006 non-null  float64
 5   indy_hh2-1100           75006 non-null  float64
 6   indy_ll2-1100           75006 non-null  float64
 7   indy_mid2-1100          75006 non-null  float64
 8   indy_macd110-440        75006 non-null  float64
 9   indy_signal110-440-110  75006 non-null  float64
 10  indy_hist_macd110-440   75006 non-null  float64
 11  indy_rsi25-ma20         75006 non-null  float64
 12  indy_6ATRTrail_DC-110   75006 non-null  float64
 13  cate_3trend-550_ma110   75006 non-null  int64  
 14  cat

In [3]:
def CreateData(dfx):
 labelCol=dfx.columns[len(dfx.columns)-1]
 
 X_df_temp = dfx.drop(columns=labelCol)
 Y_df_temp = dfx[labelCol]

 Xtemp=X_df_temp.to_numpy()
 ytemp=Y_df_temp.to_numpy()

 return X_df_temp,Y_df_temp, Xtemp,ytemp

In [4]:
#My Train
st='01-2007'
ed='03-2021'
df_train=dataset[st:ed]
#df_train.info()
X_df,Y_df,X_train,y_train=CreateData( df_train)

print(X_df.info())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75006 entries, 2007-01-03 09:45:00 to 2021-03-31 16:45:00
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             75006 non-null  float64
 1   indy_ma-1100            75006 non-null  float64
 2   indy_hh-550             75006 non-null  float64
 3   indy_ll-550             75006 non-null  float64
 4   indy_mid-550            75006 non-null  float64
 5   indy_hh2-1100           75006 non-null  float64
 6   indy_ll2-1100           75006 non-null  float64
 7   indy_mid2-1100          75006 non-null  float64
 8   indy_macd110-440        75006 non-null  float64
 9   indy_signal110-440-110  75006 non-null  float64
 10  indy_hist_macd110-440   75006 non-null  float64
 11  indy_rsi25-ma20         75006 non-null  float64
 12  indy_6ATRTrail_DC-110   75006 non-null  float64
 13  cate_3trend-550_ma110   75006 non-null  int64  
 14  cat

In [5]:
X_df.tail(5)

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2021-03-31 15:45:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.39,1.18,1.20,47.74,1.0,3,0,3,4
2021-03-31 16:00:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.45,1.16,1.29,47.00,1.0,3,0,3,4
2021-03-31 16:15:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.53,1.14,1.39,46.28,1.0,3,0,3,4
2021-03-31 16:30:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.58,1.12,1.46,46.08,1.0,3,0,3,4
2021-03-31 16:45:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.61,1.11,1.51,45.92,1.0,3,0,3,4


In [6]:
Y_df.tail(5)

datetime
2021-03-31 15:45:00    0
2021-03-31 16:00:00    0
2021-03-31 16:15:00    0
2021-03-31 16:30:00    0
2021-03-31 16:45:00    0
Name: LongSignal, dtype: int64

In [7]:
Y_df.value_counts()

0    44094
1    30912
Name: LongSignal, dtype: int64

# Multiple Tune  n_estimator and LearningRate

In [8]:
algoName='xgb'

#scoreX="neg_log_loss"
scoreX='accuracy'

xDepth=8

splitN=10
isShuffle=False  # if True  set random_state=xseed else None

#For Actual
estimatorList=list(range(40, 240,20))
learning_rateList =[0.001,0.005,0.01,0.05,0.1,0.5,1]


#For Test
# estimatorList=list(range(50, 101,50))
# learning_rateList =[0.1,0.5]

if algoName=='lgb':
 nLeafList=[31] 
 nLeafList.extend(range(50,150,50))
 print(nLeafList)




print(algoName)
print('the number of trees: ',estimatorList)
print('learning rate: ',learning_rateList)
print('depth-tree: ', xDepth)
 

xgb
the number of trees:  [40, 60, 80, 100, 120, 140, 160, 180, 200, 220]
learning rate:  [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
depth-tree:  8


# XGBClassifier change paramter consistently

In [25]:
if isShuffle==False:
    xseed=None
else:
    xseed=1990

option=1
if option==1 :
 kfold =  TimeSeriesSplit(n_splits=splitN,max_train_size=None)
elif option==2:
 kfold = KFold(n_splits=splitN, shuffle=isShuffle, random_state=xseed)
else:
 kfold = StratifiedKFold(n_splits=splitN, shuffle=isShuffle, random_state=xseed)
print(kfold)

In [26]:
if algoName=='xgb':
 param_grid = dict(learning_rate=learning_rateList, n_estimators=estimatorList)
 model = XGBClassifier(max_depth=xDepth,importance_type='gain')
elif algoName=='lgb':
 param_grid = dict(learning_rate=learning_rateList, n_estimators=estimatorList,num_leaves=nLeafList)
 #model=LGBMClassifier(max_depth=mxDepth)
 model=LGBMClassifier()

print(param_grid)
model


{'learning_rate': [0.1, 0.5], 'n_estimators': [50, 100]}


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=8,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)

In [27]:
t_Start=time.time()
print(datetime.now())

2021-04-14 00:16:00.561174


In [31]:
grid_search = GridSearchCV(model, param_grid, scoring=scoreX, n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
grid_result

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=2),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=8, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
          

In [32]:
grid_result.cv_results_

{'mean_fit_time': array([5.48266661, 9.99684858, 5.02076006, 8.30453885]),
 'std_fit_time': array([1.74411976, 3.33331323, 1.47370911, 3.1121577 ]),
 'mean_score_time': array([0.08890772, 0.13253343, 0.07805192, 0.1326319 ]),
 'std_score_time': array([0.00313616, 0.01384771, 0.00771916, 0.03489435]),
 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.5, 0.5],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[50, 100, 50, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'learning_rate': 0.1, 'n_estimators': 50},
  {'learning_rate': 0.1, 'n_estimators': 100},
  {'learning_rate': 0.5, 'n_estimators': 50},
  {'learning_rate': 0.5, 'n_estimators': 100}],
 'split0_test_score': array([0.8687305 , 0.8738101 , 0.85761139, 0.85821134]),
 'split1_test_score': array([0.81553476, 0.81769458, 0.82613391, 0.82421406]),
 'mean_test_scor

In [33]:
print(datetime.now())
t_End=time.time()
t_elapsed=(t_End-t_Start)/60/60
print('Total execute train : ',round(t_elapsed,2))

2021-04-14 00:17:55.601063
Total execute train :  0.03


# Summarize best results

In [34]:
print(param_grid)
print("Best: %f using %s" % ( grid_result.best_score_, grid_result.best_params_))



{'learning_rate': [0.1, 0.5], 'n_estimators': [50, 100]}
Best: 0.845752 using {'learning_rate': 0.1, 'n_estimators': 100}


In [35]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
                                 
for mean, stdev, param in zip(means, stds, params):
    print("%f,%f,%r" % (mean, stdev, param))
                                
                

0.842133,0.026598,{'learning_rate': 0.1, 'n_estimators': 50}
0.845752,0.028058,{'learning_rate': 0.1, 'n_estimators': 100}
0.841873,0.015739,{'learning_rate': 0.5, 'n_estimators': 50}
0.841213,0.016999,{'learning_rate': 0.5, 'n_estimators': 100}


# Plot Result

In [36]:
# scores = np.array(means).reshape(len(learning_rateList), len(estimatorList))
# len(scores)

2

In [37]:
# buildtime=datetime.now().strftime('%d%m%y_%H%M')
# buildtime

'140421_0024'

In [40]:
# for i, value in enumerate(learning_rateList):
#   pyplot.plot(estimatorList, scores[i], label='learning_rate: ' + str(value))
# pyplot.legend()
# pyplot.xlabel('n_estimators')
# pyplot.ylabel(scoreX)
# pyplot.savefig(f'nTree_vs_lR_{st}To{ed}_{scoreX}-{algoName}_{buildtime}.png')